In [ ]:
import os
import numpy as np
import pandas as pd
import random
import seaborn as sns

import datetime as datetime
import matplotlib.dates as dates
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from contextlib import contextmanager
from time import time
from tqdm import tqdm
import lightgbm as lgbm

from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

# data : osaka_summary.csv

In [ ]:
data00 = pd.read_csv("../input/covid19-osaka/osaka_summary - 0814.csv")
data00[-5:]

In [ ]:
data0=data00[['date','examined positives']]

In [ ]:
data0['ratio to 7 days before']=1

In [ ]:
n=len(data0)
for i in range(n):    
    cpi=data0['examined positives'][i]
    data0.loc[i+7,'positives 7 days before']=cpi
data0[100:114]

In [ ]:
data0['ratio to 7 days before']=data0['examined positives']*100/data0['positives 7 days before']
data0['mean ratio to 7 days before']=data0['ratio to 7 days before'].rolling(window=7).mean()
data1=data0[400:-7]
data1

In [ ]:
fig=make_subplots(specs=[[{"secondary_y":False}]])
fig.add_trace(go.Scatter(x=data1['date'],y=data1['mean ratio to 7 days before'],name='mean ratio weekly'),secondary_y=False,)
fig.update_layout(autosize=False,width=700,height=500,title_text="Mean ratio of increase weekly in Osaka")
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Ratio",secondary_y=False)
fig.show()

In [ ]:
data2=data1[-7:]
data2

In [ ]:
factor=round(data2.loc[data2.index.tolist()[-1],'mean ratio to 7 days before'])/100
print(factor)

In [ ]:
N=[]
for i in range(40):
    N+=[i]
NUMB=pd.DataFrame(N)

In [ ]:
data2['date2']=pd.to_datetime(data2['date'])
data2=pd.concat([data2,NUMB],axis=0)
data2=data2.drop([0,'date'],axis=1).reset_index(drop=True)
data2[0:10]

In [ ]:
data2['pcr_positives_ft']=data2['examined positives']
#data2['pcr_positives160']=data2['examined positives']
#data2['pcr_positives140']=data2['examined positives']
data2['pcr_positives110']=data2['examined positives']

In [ ]:
for i in range(7,47):
    data2.loc[i,'date2']=data2.loc[i-1,'date2']+datetime.timedelta(days=1)
    data2.loc[i,'pcr_positives_ft']=data2.loc[i-7,'pcr_positives_ft']*factor
    #data2.loc[i,'pcr_positives160']=data2.loc[i-7,'pcr_positives160']*1.6
    #data2.loc[i,'pcr_positives140']=data2.loc[i-7,'pcr_positives140']*1.4
    data2.loc[i,'pcr_positives110']=data2.loc[i-7,'pcr_positives110']*1.1    
data3=data2[['date2','examined positives','pcr_positives_ft','pcr_positives110']]
data3

In [ ]:
fig=make_subplots(specs=[[{"secondary_y":False}]])
fig.add_trace(go.Scatter(x=data3['date2'],y=data3['pcr_positives_ft'],name='the latest ratio of incresase weekly'),secondary_y=False,)
#fig.add_trace(go.Scatter(x=data3['date2'],y=data3['pcr_positives160'],name='160% incresase weekly'),secondary_y=False,)
#fig.add_trace(go.Scatter(x=data3['date2'],y=data3['pcr_positives140'],name='140% incresase weekly'),secondary_y=False,)
fig.add_trace(go.Scatter(x=data3['date2'],y=data3['pcr_positives110'],name='110% incresase weekly'),secondary_y=False,)
fig.add_trace(go.Scatter(x=data3['date2'],y=data3['examined positives'],name='actual number positives'),secondary_y=False,)
fig.update_layout(autosize=False,width=700,height=500,title_text="Prediction of number positives in Osaka")
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Number",secondary_y=False)
fig.show()

### prediction as of 2021/08/05
* reach 5,000 on 2021/08/20
* reah 10,000 on 2021/08/27

### prediction as of 2021/07/31
* reach 5,000 on 2021/08/14
* reah 10,000 on 2021/08/19